## borramos collection en mongoDB

In [1]:
from pymongo import MongoClient
#mongodb conection
client = MongoClient('localhost', 27017)
db = client.bici
collection = db.datacleaned
collection.drop()

# importamos los datos de BICIMAD
<img src="https://pbs.twimg.com/profile_images/621342633655271424/26fdqUsf.png" style="width:150px; display:inline">


In [2]:
# from urllib import urlopen
import urllib.request

import simplejson as json

#Es un api no oficial pero que ofrece datos cada 10 min

url = urllib.request.urlopen('http://bicimad-api.herokuapp.com/api-v1/locations/').read()
url = json.loads(url)

In [3]:
import pandas as pd
dataPre = url["locations"]
data_mad = pd.DataFrame(dataPre)

In [4]:
data_mad['servicio']  = 'bicimad'

In [5]:
data_mad.head()

,activo,direccion,idestacion,latitud,libres,longitud,luz,nombre,numero_estacion,porcentaje,servicio
0,True,Puerta del Sol No 1,1a,40.4168961,12,-3.7024255,False,Puerta del Sol A,1a,50.000000,bicimad
1,True,Calle Fuencarral No 108,5,40.4285280,21,-3.7020599,False,Fuencarral,5,77.777778,bicimad
2,True,Calle Marques de la Ensenada No 16,10,40.4250863,13,-3.6918807,False,Marques de la Ensenada,10,54.166667,bicimad
3,True,Calle San Vicente Ferrer No 64,15,40.4261649,15,-3.7073764,False,S.Vicente Ferrer,15,71.428571,bicimad
4,True,Calle Alcala No 49,20a,40.4192342,12,-3.6954615,False,Banco de España A,20a,50.000000,bicimad


# importamos los datos de BICING
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/a/a1/Bicing_logo.svg/200px-Bicing_logo.svg.png" style="width:150px; display:inline">


In [6]:
url_bcn = urllib.request.urlopen('http://wservice.viabicing.cat/v2/stations').read()
url_bcn = url_bcn.split(b'\n')
# url_bcn = url_bcn[1:len(url_bcn)]
url_bcn = eval(url_bcn[0])
url_bcn = url_bcn["stations"]
print((url_bcn))
data_bcn = pd.DataFrame(url_bcn)

[{'altitude': '21', 'slots': '12', 'streetNumber': '760', 'nearbyStations': '24, 369, 387, 426', 'type': 'BIKE', 'streetName': 'Gran Via Corts Catalanes', 'bikes': '10', 'latitude': '41.397952', 'status': 'OPN', 'id': '1', 'longitude': '2.180042'}, {'altitude': '21', 'slots': '24', 'streetNumber': '126', 'nearbyStations': '360, 368, 387, 414', 'type': 'BIKE', 'streetName': 'Roger de Flor\\/ Gran Vía', 'bikes': '1', 'latitude': '41.39553', 'status': 'OPN', 'id': '2', 'longitude': '2.17706'}, {'altitude': '21', 'slots': '15', 'streetNumber': '44', 'nearbyStations': '4, 6, 119, 419', 'type': 'BIKE', 'streetName': 'Ali Bei', 'bikes': '11', 'latitude': '41.393699', 'status': 'OPN', 'id': '3', 'longitude': '2.181137'}, {'altitude': '21', 'slots': '13', 'streetNumber': '13', 'nearbyStations': '3, 5, 359, 419', 'type': 'BIKE', 'streetName': 'Ribes', 'bikes': '6', 'latitude': '41.39347', 'status': 'OPN', 'id': '4', 'longitude': '2.18149'}, {'altitude': '16', 'slots': '28', 'streetNumber': '11',

In [7]:
data_bcn['servicio']  = 'bicing'

In [8]:
data_bcn['direccion'] = data_bcn["streetName"] + data_bcn["streetNumber"]
data_bcn['slots'] = data_bcn["bikes"].astype(float) + data_bcn["slots"].astype(float)
data_bcn['porcenjate'] = data_bcn['bikes'].astype(float) / data_bcn['slots'].astype(float)


del data_bcn["streetName"]
del data_bcn["streetNumber"]
del data_bcn["nearbyStations"]
del data_bcn["altitude"]
del data_bcn["type"]

In [9]:
del data_mad["luz"]
data_mad['slots'] = data_mad['libres'] / data_mad['porcentaje'] *100
del data_mad['porcentaje']
data_mad['porcentaje'] = data_mad['libres'].astype(float) / data_mad['slots'].astype(float)
del data_mad['numero_estacion']
del data_mad['nombre']

In [10]:
data_bcn.columns 

Index(['bikes', 'id', 'latitude', 'longitude', 'slots', 'status', 'servicio',
       'direccion', 'porcenjate'],
      dtype='object')

In [11]:
data_mad.columns

Index(['activo', 'direccion', 'idestacion', 'latitud', 'libres', 'longitud',
       'servicio', 'slots', 'porcentaje'],
      dtype='object')

In [12]:
data_bcn.head()

,bikes,id,latitude,longitude,slots,status,servicio,direccion,porcenjate
0,10,1,41.397952,2.180042,22.0,OPN,bicing,Gran Via Corts Catalanes760,0.454545
1,1,2,41.39553,2.17706,25.0,OPN,bicing,Roger de Flor\/ Gran Vía126,0.040000
2,11,3,41.393699,2.181137,26.0,OPN,bicing,Ali Bei44,0.423077
3,6,4,41.39347,2.18149,19.0,OPN,bicing,Ribes13,0.315789
4,10,5,41.391075,2.180223,38.0,OPN,bicing,Pg Lluís Companys11,0.263158


In [13]:
data_bcn.columns = ['libres', 'id_estacion', 'latitud', 'longitud', 'slots', 'status', 'servicio', 'direccion','porcentaje']
data_mad.columns = ['status', 'direccion', 'id_estacion', 'latitud', 'libres', 'longitud','servicio', 'slots', 'porcentaje']

In [14]:
data_mad=data_mad.dropna() 
data_bcn=data_bcn.dropna() 

In [15]:
import warnings
warnings.filterwarnings("ignore")

In [16]:
data_mad['disponibilidad'] = 'baja disponibilidad'
data_mad['disponibilidad'][(data_mad['porcentaje'] > 0.20) & (data_mad['porcentaje'] <= 0.5)] = 'disponibilidad media'
data_mad['disponibilidad'][(data_mad['porcentaje'] > 0.50) & (data_mad['porcentaje'] <= 0.8)] = 'alta disponibilidad'
data_mad['disponibilidad'][data_mad['porcentaje'] >0.80] = 'muy alta disponibilidad'


In [17]:
data_bcn['disponibilidad'] = 'baja disponibilidad'
data_bcn['disponibilidad'][(data_bcn['porcentaje'] > 0.20) & (data_bcn['porcentaje'] <= 0.5)] = 'disponibilidad media'
data_bcn['disponibilidad'][(data_bcn['porcentaje'] > 0.50) & (data_bcn['porcentaje'] <= 0.8)] = 'alta disponibilidad'
data_bcn['disponibilidad'][data_bcn['porcentaje'] >0.80] = 'muy alta disponibilidad'



In [18]:
data_mad.head()

,status,direccion,id_estacion,latitud,libres,longitud,servicio,slots,porcentaje,disponibilidad
0,True,Puerta del Sol No 1,1a,40.4168961,12,-3.7024255,bicimad,24.0,0.500000,disponibilidad media
1,True,Calle Fuencarral No 108,5,40.4285280,21,-3.7020599,bicimad,27.0,0.777778,alta disponibilidad
2,True,Calle Marques de la Ensenada No 16,10,40.4250863,13,-3.6918807,bicimad,24.0,0.541667,alta disponibilidad
3,True,Calle San Vicente Ferrer No 64,15,40.4261649,15,-3.7073764,bicimad,21.0,0.714286,alta disponibilidad
4,True,Calle Alcala No 49,20a,40.4192342,12,-3.6954615,bicimad,24.0,0.500000,disponibilidad media


In [19]:
data_bcn.head()

,libres,id_estacion,latitud,longitud,slots,status,servicio,direccion,porcentaje,disponibilidad
0,10,1,41.397952,2.180042,22.0,OPN,bicing,Gran Via Corts Catalanes760,0.454545,disponibilidad media
1,1,2,41.39553,2.17706,25.0,OPN,bicing,Roger de Flor\/ Gran Vía126,0.040000,baja disponibilidad
2,11,3,41.393699,2.181137,26.0,OPN,bicing,Ali Bei44,0.423077,disponibilidad media
3,6,4,41.39347,2.18149,19.0,OPN,bicing,Ribes13,0.315789,disponibilidad media
4,10,5,41.391075,2.180223,38.0,OPN,bicing,Pg Lluís Companys11,0.263158,disponibilidad media


# importamos los datos de BILBON BIZI
<img src="http://www.bilbao.net/Agenda21/agenda21cont/bilbon_bizi.jpg" style="width:150px; display:inline">


In [20]:
import pandas as pd
data_bio = pd.read_csv("bilbaoBICI.csv", sep=';', encoding = "latin-1")
data_bio['libres'] = data_bio['BLIBRES']
data_bio['slots'] = data_bio['ALIBRES'] + data_bio['AAVERIADOS'] + data_bio['AUSADOS']
data_bio['porcentaje'] = data_bio['libres'] / data_bio['slots'] *100
data_bio['disponibilidad'] = 'baja disponibilidad'
data_bio['disponibilidad'][(data_bio['porcentaje'] > 0.20) & (data_bio['porcentaje'] <= 0.5)] = 'disponibilidad media'
data_bio['disponibilidad'][(data_bio['porcentaje'] > 0.50) & (data_bio['porcentaje'] <= 0.8)] = 'alta disponibilidad'
data_bio['disponibilidad'][data_bio['porcentaje'] >0.80] = 'muy alta disponibilidad'
data_bio['servicio']='bilbonbizi'
data_bio.head()

,AAVERIADOS,ALIBRES,AUSADOS,BAVERIADAS,BLIBRES,ESTADO,ID,LATITUD,LONGITUD,NOMBRE,libres,slots,porcentaje,disponibilidad,servicio
0,1,12,2,0,2,COMUNICA,3001,43.281575,-2.962019,01-PLAZA LEVANTE,2,15,13.333333,muy alta disponibilidad,bilbonbizi
1,2,10,8,0,8,COMUNICA,3002,43.269819,-2.949229,02-IRUÑA,8,20,40.000000,muy alta disponibilidad,bilbonbizi
2,0,9,11,0,11,COMUNICA,3003,43.264402,-2.922888,03-AYUNTAMIENTO,11,20,55.000000,muy alta disponibilidad,bilbonbizi
3,2,24,9,0,9,COMUNICA,3004,43.259518,-2.924383,04-PLAZA ARRIAGA,9,35,25.714286,muy alta disponibilidad,bilbonbizi
4,0,9,6,0,6,COMUNICA,3005,43.250165,-2.930786,05-SANTIAGO COMPOSTELA,6,15,40.000000,muy alta disponibilidad,bilbonbizi


In [21]:
del data_bio["BAVERIADAS"]
del data_bio["AAVERIADOS"]
del data_bio["ALIBRES"]
del data_bio["AUSADOS"]
del data_bio["BLIBRES"]
# data_bio['status'] = data_bio["ESTADO"]
data_bio.columns = ['status', 'id_estacion', 'latitud', 'longitud', 'direccion', 'libres','slots', 'porcentaje', 'disponibilidad', 'servicio']
data_bio.head()


,status,id_estacion,latitud,longitud,direccion,libres,slots,porcentaje,disponibilidad,servicio
0,COMUNICA,3001,43.281575,-2.962019,01-PLAZA LEVANTE,2,15,13.333333,muy alta disponibilidad,bilbonbizi
1,COMUNICA,3002,43.269819,-2.949229,02-IRUÑA,8,20,40.000000,muy alta disponibilidad,bilbonbizi
2,COMUNICA,3003,43.264402,-2.922888,03-AYUNTAMIENTO,11,20,55.000000,muy alta disponibilidad,bilbonbizi
3,COMUNICA,3004,43.259518,-2.924383,04-PLAZA ARRIAGA,9,35,25.714286,muy alta disponibilidad,bilbonbizi
4,COMUNICA,3005,43.250165,-2.930786,05-SANTIAGO COMPOSTELA,6,15,40.000000,muy alta disponibilidad,bilbonbizi


## guardamos en mongoDB

In [22]:
from pymongo import MongoClient
#mongodb conection
client = MongoClient('localhost', 27017)
db = client.bici
collection = db.datacleaned
collection.insert_many(data_mad.to_dict("records"))
collection.insert_many(data_bcn.to_dict("records"))
collection.insert_many(data_bio.to_dict("records"))